In [2]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.path.join(os.getcwd(), ".."), ".."))
sys.path.append(parent_dir)

import numpy as np
import time
import matplotlib.pyplot as plt
from data_generation.models.tech_substitution import TechnologySubstitution, NumericalSolver

In [3]:
def run_performance_test(n_samples, num_steps, seed=42):
    """
    Compare performance of RK4 vs solve_ivp integration methods
    """
    np.random.seed(seed)
    
    # Initialize model and solver
    model = TechnologySubstitution()
    solver = NumericalSolver(model)
    
    # Test parameters
    delta_t = 0.1
    constant_value = 0.8  # Control less than 1
    
    # Create initial conditions
    X = np.ones((n_samples, 2))
    X += np.random.uniform(0.1, 0.5, X.shape)
    
    # Constant control
    control = np.full((num_steps, n_samples, 1), constant_value)
    
    # Test RK4 method
    start_time = time.time()
    trajectory_rk4 = solver.step(X, control, delta_t, num_steps, steady_control=False)
    rk4_time = time.time() - start_time
    
    # Test solve_ivp method
    start_time = time.time()
    trajectory_ivp = solver.step(X, control, delta_t, num_steps, steady_control=True)
    ivp_time = time.time() - start_time
    
    # Calculate max difference
    max_diff = np.max(np.abs(trajectory_rk4 - trajectory_ivp))
    
    print(f"\nTest: n_samples={n_samples}, num_steps={num_steps}")
    print(f"{'='*50}")
    print(f"RK4 time:      {rk4_time:.3f} s")
    print(f"solve_ivp time: {ivp_time:.3f} s")
    print(f"Speed ratio:    {rk4_time/ivp_time:.2f}x (RK4/IVP)")
    print(f"Max difference: {max_diff:.2e}")

In [4]:
# Test with increasing sample sizes to see scaling of differences
sample_sizes = [100, 1000]
timesteps = [1, 10, 50]

for n_samples in sample_sizes:
    print(f"\nTesting with {n_samples} samples:")
    print(f"{'='*50}")
    for steps in timesteps:
        run_performance_test(n_samples=n_samples, num_steps=steps)


Testing with 100 samples:

Test: n_samples=100, num_steps=1
RK4 time:      1.260 s
solve_ivp time: 0.693 s
Speed ratio:    1.82x (RK4/IVP)
Max difference: 5.03e-10

Test: n_samples=100, num_steps=10
RK4 time:      3.477 s
solve_ivp time: 1.127 s
Speed ratio:    3.09x (RK4/IVP)
Max difference: 2.93e-07

Test: n_samples=100, num_steps=50
RK4 time:      17.225 s
solve_ivp time: 1.701 s
Speed ratio:    10.13x (RK4/IVP)
Max difference: 3.95e-04

Test: n_samples=100, num_steps=100
RK4 time:      36.313 s
solve_ivp time: 2.503 s
Speed ratio:    14.51x (RK4/IVP)
Max difference: 1.01e-03

Testing with 1000 samples:

Test: n_samples=1000, num_steps=1
RK4 time:      2.956 s
solve_ivp time: 5.958 s
Speed ratio:    0.50x (RK4/IVP)
Max difference: 5.93e-10

Test: n_samples=1000, num_steps=10
RK4 time:      31.497 s
solve_ivp time: 10.882 s
Speed ratio:    2.89x (RK4/IVP)
Max difference: 3.05e-07

Test: n_samples=1000, num_steps=50
RK4 time:      146.813 s
solve_ivp time: 14.781 s
Speed ratio:    9.

In [5]:
# High sample size, one step only, e.g. as if used when control is changing
run_performance_test(n_samples=10000, num_steps=1)


Test: n_samples=10000, num_steps=1
RK4 time:      28.543 s
solve_ivp time: 57.971 s
Speed ratio:    0.49x (RK4/IVP)
Max difference: 1.17e-09


In [ ]:
# very long trajectory, inaccuracies in rk4 
# #NOTE: ground truth is not known but assuemd that solve_ivp is more accurate
# Performance of rk4 scales very badly with high num_steps, too
# solve_ivp scales extremely well with num_steps
# NOTE: If every performance becomes an issue in sampling we might want to scale with 
run_performance_test(n_samples=100, num_steps=1000)


Test: n_samples=100, num_steps=1000
RK4 time:      536.893 s
solve_ivp time: 6.257 s
Speed ratio:    85.81x (RK4/IVP)
Max difference: 2.66e-03


In [7]:
def run_solve_ivp_test(n_samples, num_steps, seed=42):
    """
    Test solve_ivp performance with sample-specific controls
    """
    np.random.seed(seed)
    
    # Initialize model and solver
    model = TechnologySubstitution()
    solver = NumericalSolver(model)
    
    # Test parameters
    delta_t = 0.1
    
    # Create initial conditions
    X = np.ones((n_samples, 2))
    X += np.random.uniform(0.1, 0.5, X.shape)
    
    # Create different control for each sample
    # Using uniform distribution between 0.5 and 0.9 for controls
    sample_controls = np.random.uniform(0.5, 0.9, (n_samples, 1))
    control = np.tile(sample_controls, (num_steps, 1, 1))
    
    # Run solve_ivp
    start_time = time.time()
    trajectory = solver.step(X, control, delta_t, num_steps, steady_control=True)
    solve_time = time.time() - start_time
    
    print(f"\nTest: n_samples={n_samples}, num_steps={num_steps}")
    print(f"Solve time: {solve_time:.3f} s")
    print(f"Time per data point: {solve_time/n_samples/num_steps*1000:.2f} ms")

In [8]:
sample_sizes = [100, 1000]
timesteps = [10, 50, 100, 1000]

print("Running solve_ivp performance tests...")
for steps in timesteps:
    print(f"\nNumber of timesteps: {steps}")
    print("="*40)
    for n_samples in sample_sizes:
        run_solve_ivp_test(n_samples=n_samples, num_steps=steps)

Running solve_ivp performance tests...

Number of timesteps: 10

Test: n_samples=100, num_steps=10
Solve time: 1.004 s
Time per data point: 1.00 ms

Test: n_samples=1000, num_steps=10
Solve time: 8.979 s
Time per data point: 0.90 ms

Number of timesteps: 50

Test: n_samples=100, num_steps=50
Solve time: 1.769 s
Time per data point: 0.35 ms

Test: n_samples=1000, num_steps=50
Solve time: 14.569 s
Time per data point: 0.29 ms

Number of timesteps: 100

Test: n_samples=100, num_steps=100
Solve time: 2.401 s
Time per data point: 0.24 ms

Test: n_samples=1000, num_steps=100
Solve time: 20.668 s
Time per data point: 0.21 ms

Number of timesteps: 1000

Test: n_samples=100, num_steps=1000
Solve time: 5.203 s
Time per data point: 0.05 ms

Test: n_samples=1000, num_steps=1000
Solve time: 43.969 s
Time per data point: 0.04 ms
